Method to transfer pipeline for other python files to use

In [1]:
import joblib

def transfer_pipeline(pipeline):
    return joblib.dump(pipeline, 'pipeline_for_app.pkl')

Drop ID column

Drop NA

Drop "Other"

Drop age less than 21 because the questions presented to make the predicted dont apply to them

Scale

SMOTENC

In [2]:
import pandas as pd
data = pd.read_csv("healthcare-dataset-stroke-data.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


Train test split

Drop NA

In [3]:
data = data.dropna()

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4909 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 4909 non-null   int64  
 1   gender             4909 non-null   object 
 2   age                4909 non-null   float64
 3   hypertension       4909 non-null   int64  
 4   heart_disease      4909 non-null   int64  
 5   ever_married       4909 non-null   object 
 6   work_type          4909 non-null   object 
 7   Residence_type     4909 non-null   object 
 8   avg_glucose_level  4909 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     4909 non-null   object 
 11  stroke             4909 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 498.6+ KB


Drop ID

In [4]:
data = data.drop(columns= "id")

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4909 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             4909 non-null   object 
 1   age                4909 non-null   float64
 2   hypertension       4909 non-null   int64  
 3   heart_disease      4909 non-null   int64  
 4   ever_married       4909 non-null   object 
 5   work_type          4909 non-null   object 
 6   Residence_type     4909 non-null   object 
 7   avg_glucose_level  4909 non-null   float64
 8   bmi                4909 non-null   float64
 9   smoking_status     4909 non-null   object 
 10  stroke             4909 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 460.2+ KB


Drop anyone younger than 21 years old

In [5]:
data = data[data.age > 20]
print("Lowest age:", data["age"].min())
data.info()

Lowest age: 21.0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3909 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             3909 non-null   object 
 1   age                3909 non-null   float64
 2   hypertension       3909 non-null   int64  
 3   heart_disease      3909 non-null   int64  
 4   ever_married       3909 non-null   object 
 5   work_type          3909 non-null   object 
 6   Residence_type     3909 non-null   object 
 7   avg_glucose_level  3909 non-null   float64
 8   bmi                3909 non-null   float64
 9   smoking_status     3909 non-null   object 
 10  stroke             3909 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 366.5+ KB


Drop "Other" in gender field

In [6]:
data = data.drop(data.index[data["gender"] == "Other"])

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3908 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             3908 non-null   object 
 1   age                3908 non-null   float64
 2   hypertension       3908 non-null   int64  
 3   heart_disease      3908 non-null   int64  
 4   ever_married       3908 non-null   object 
 5   work_type          3908 non-null   object 
 6   Residence_type     3908 non-null   object 
 7   avg_glucose_level  3908 non-null   float64
 8   bmi                3908 non-null   float64
 9   smoking_status     3908 non-null   object 
 10  stroke             3908 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 366.4+ KB


In [ ]:
from sklearn.model_selection import train_test_split

data_train, data_test = train_test_split(data, train_size = 0.8, random_state=1)

X = data_train.drop('stroke', axis = 1)
y = data_train['stroke']

Random Over Sampler for now (Change to smotenc) 

In [ ]:
from imblearn.over_sampling import RandomOverSampler

over_sampler = RandomOverSampler()
over_X, over_y = over_sampler.fit_resample(X, y)

Scaling data

Imblearn first then scale

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

cat_columns = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
num_columns = ['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi']

one_hot_encoder = OneHotEncoder(sparse_output=False)
one_hot_encoder.fit_transform(data_train[cat_columns])

standard_scaler = StandardScaler()
standard_scaler.fit_transform(data_train[num_columns])

Pipeline

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


num_pipeline = Pipeline([
    ("scale", StandardScaler())
])

one_hot_pipeline = Pipeline([
    ("one hot encode", OneHotEncoder())
])

preprocessing = ColumnTransformer([
    ("Numeric", num_pipeline, num_columns),
    ("One-Hot", one_hot_pipeline, cat_columns),
])


Pickle the pipeline

In [ ]:
#transfer_pipeline(preprocessing)